In [1]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import playsound
import argparse
import imutils
import time
import dlib
import cv2

In [3]:
def sound_alarm(path):
    playsound.playsound(path)
path = "C:/Users/91874/Downloads/code-red-185448.mp3"

In [4]:
#let's define EAR----->
def Eye_Aspect_Ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
  
        C = dist.euclidean(eye[0], eye[3])
        
        ear = (A + B) / (2.0 * C)
    
        return ear

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"C:\Users\91874\Downloads\shape_predictor_dlib.dat")

In [12]:
threshold_value=0.3
allowed_frames_for_lower_ear = 48
COUNTER=0
ALARM_ON = False

In [7]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [8]:
from threading import Thread

In [ ]:
vs = VideoStream(src=0).start()
time.sleep(1.0)
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = Eye_Aspect_Ratio(leftEye)
        rightEAR = Eye_Aspect_Ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0
        
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        if ear < threshold_value:
            COUNTER += 1
            if COUNTER >= allowed_frames_for_lower_ear:
                if not ALARM_ON:
                    ALARM_ON = True  
                    if path != "":
                        t = Thread(target=sound_alarm, args=(path,))
                        t.daemon = True
                        t.start()
                cv2.putText(frame, "!!!!!!DROWSINESS ALERT!!!!!!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            COUNTER = 0
            if ALARM_ON:
                ALARM_ON = False
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    
    if key == ord("q"):
            break

cv2.destroyAllWindows()
vs.stop()